<a href="https://colab.research.google.com/github/dab0012/olivia/blob/master/Olivia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OLIVIA
**Open-source Library Indexes Vulnerability Identification and Analysis**

The use of centralized library repositories to reduce development times and costs is universal, in virtually all languages and types of software projects. Due to the transitivity of dependencies, the appearance of a single defect in the repository can have extensive and difficult-to-predict effects on the ecosystem. These defects cause functional errors or performance or security problems. The risk is difficult to grasp for developers, who only explicitly import a small part of the dependencies.

OLVIA uses an approach based on the vulnerability of the dependency network of software packages, which measures how sensitive the repository is to the random introduction of defects. The goals of the model are  to contribute to the understanding of propagation mechanisms of software defects and to study feasible protection strategies. This can benefit multiple parties:

* **Centralised package managers**, to establish policies and manual or automatic control processes that improve the security and stability of the repositories.
* **Software developers** in general, to assess the different risks introduced by the dependencies used in their projects, and **package developers** in particular to understand their responsibility on the ecosystem.
* Developers of **continuous quality tools**, to define the concept of vulnerability based on the modeling of the network of package dependencies.
---

**Author**: Daniel Setó Rey

https://github.com/dsr0018/olivia

**License**: Olivia and this notebook are published under a MIT [license](https://github.com/dsr0018/olivia/blob/master/LICENSE). The information of dependencies has been obtained from the libraries.io [data snapshots](https://libraries.io/data) (by Tidelift).

---

*This notebook is part of a user guide series that cover in detail the operation of the library.*
*This time we treat the basic operations of loading/creating models and dealing with package properties and metrics.*



## Setup runtime environment

In [ ]:
from google.colab import drive

# Mount Gdrive
drive.mount("/content/gdrive")

# Prepare test folders
!mkdir "/content/gdrive/My Drive/test_olivia"
%cd "/content/gdrive/My Drive/test_olivia"

# Download source code from github
!git clone https://github.com/dab0012/olivia
%cd olivia

# Install requeriments
! pip install -r requirements.txt


## A - Basic model usage

[01 - Load model](#01---Load-model)&ensp;|&ensp;[02 - Package properties](#02---Package-properties)&ensp;|&ensp;[03 - Package metrics](#03---Package-metrics)&ensp;|&ensp;[04 - Custom models](#04---Custom-models)

*OliviaNetwork* is essentially a directed graph with some additional structures to facilitate working with metrics in large dependency networks. The model can be built from a NetworkX directed network or from a file in adjacency list format.

### 01 - Load model

Load a pre-built model from file. This one is based in a snapshot of the Python Package Index (https://pypi.org/) from 2020-01 (data from https://libraries.io/):

In [ ]:
from olivia.model import OliviaNetwork

In [ ]:
pypi = OliviaNetwork(r'data/pypi-2020-01-12.olv')

As expected, *len()* returns the number of the packages in the network:

In [ ]:
len(pypi)

You may iterate over package names:

In [ ]:
# Packages in PyPI starting with 'zai'
for package in pypi:
    if package[0:3]=='zai':
        print(package)

### 02 - Package properties

Access via *getitem* returns a special view object:

In [ ]:
pypi['networkx']

*PackageInfoView* contains methods returning stats for specific packages. For example, direct dependants are other packages that import networkx in their code:

In [ ]:
print(f"NetworkX has {len(pypi['networkx'].direct_dependants())} direct dependants")  

Packages may depend on NetworkX not only directly, but also via transitive dependencies:

![Dependants and transitive dependants](docs/img/dependants.png "Transitive dependencies")  
<br>

In [ ]:
print(f"NetworkX has {len(pypi['networkx'].transitive_dependants())} transitive dependants (includes direct dependants)")

Packages are returned as sets so we may apply the usual set operators. For example, these packages depend on NetworkX but do not explicitly import it:

In [ ]:
print(pypi['networkx'].transitive_dependants() - pypi['networkx'].direct_dependants())

Lets pick one of them and check its dependencies, the packages on which it depends:

In [ ]:
pypi['nobrainer'].direct_dependencies()

NetworkX is not a direct dependency but a transitive one:

In [ ]:
pypi['nobrainer'].transitive_dependencies()

### 03 - Package metrics

**REACH** of a package *u* is the number of transitive dependents of *u* plus 1.

In [ ]:
pypi['networkx'].reach()

REACH represents the number of packages in the network that could be affected by the occurrence of a defect in *u*, like a bug or a security vulnerability. A bug in networkx could affect 590 packages, including NetworkX.

You may calculate REACH package by package, as in the previous example. However, this involves many redundant computations and is very slow. OLIVIA provides efficient methods to calculate REACH for all the nodes in the network.

In [ ]:
from olivia.packagemetrics import Reach
pypi_reach = pypi.get_metric(Reach)

*OliviaModel.get_metric(...)* returns a *MetricStats* object with the results of the computation. *get_metric(...)* accepts as parameter classes implementing the *compute()* method, such as the ones in *olivia.packagemetrics*

In [ ]:
pypi_reach

In [ ]:
pypi_reach['networkx']

Once calculated through *get_metric*, the *MetricStats* object is cached into the *OliviaNetwork* model. In this way, other complex algorithms that use large metric results are freed from managing each one on their own. 

So there is really no need to store the results into an independent variable like we did.

In [ ]:
pypi_reach = pypi.get_metric(Reach)

In [ ]:
pypi.get_metric(Reach)['networkx']

The management of the cache is semi-automatic. You can request a value from a network-wide metric that has not yet been calculated and it will be computed and cached the first time:

In [ ]:
from olivia.packagemetrics import Impact

%time pypi.get_metric(Impact)['networkx']

In [ ]:
%time pypi.get_metric(Impact)['networkx']

In [ ]:
pypi.get_metric(Impact)['numpy']

In [ ]:
pypi.get_metric(Impact)['matplotlib']

By the way, **IMPACT** is an alternative way of measuring the effect of a defect appearing in the network. It corresponds to the number of "links" affected (the number of "imports" in Python terms), it could be a better measure of the effort required to recover the network. Technically speaking it is the number of arcs in the graph induced by a node and its transitive dependents.

On the other hand, **SURFACE** is the size of the set of transitive dependencies plus 1. SURFACE(*u*) is the number of packages whose defects could affect *u*. High SURFACE packages are more vulnerable to random failures.

In [ ]:
from olivia.packagemetrics import Surface

pypi.get_metric(Surface)['pandas']

![Package metrics](docs/img/pmetrics.png "Olivia package metrics")
<br>

*MetricStats* is not just about storing values. It also has some basic methods that are useful for working with metrics. For example, you can get top and bottom packages according to the metric value:

In [ ]:
pypi_reach.top(10)

In [ ]:
pypi_reach.bottom()

In [ ]:
pypi.get_metric(Surface).top(10)

As *MetricStats* implements arithmetic operators, you may define compound metrics or other operations like corrections or normalization:

In [ ]:
normalized_reach = pypi.get_metric(Reach)/len(pypi)
normalized_reach.top(10)

and there we can see that a failure in *six* could affect more than 40% of the packets in the network.

Likewise, if we normalize SURFACE in relation to the size of the network, we obtain the probability that a uniformly random failure will affect each package:


In [ ]:
(pypi.get_metric(Surface)/len(pypi)).top(10)

Some examples of compound metrics:

In [ ]:
from olivia.packagemetrics import DependentsCount

mean_degree = pypi.get_metric(DependentsCount).values.mean()
degree_divergence = (pypi.get_metric(DependentsCount)-mean_degree)**2
degree_divergence.top(5)

In [ ]:
# Impact / Reach ratio
(pypi.get_metric(Impact)/pypi.get_metric(Reach)).top(5)

### 04 - Custom models

An *OliviaNetwork* model can be built from a text file with package dependencies in adjacency list format. You may provide compressed GZIP or BZ2 files.

In [ ]:
net = OliviaNetwork()
net.build_model(r'data/pypi-dependencies-net-2020-01-12.bz2')

In [ ]:
net.get_metric(Reach).top(10)

 *build_model(...)* also accepts an arbitrary NetworkX directed network as input:

In [ ]:
import networkx as nx
net = OliviaNetwork()
net.build_model(nx.path_graph(5, create_using=nx.DiGraph))

In [ ]:
net.get_metric(Reach).top(5)

'network' property is a reference to a NetworkX (https://networkx.org/) DiGraph object representing the dependency structure:

Once built, models may be saved with *OliviaNetwork.save(filename)* method. Files are GZIP compressed and also store the cached metrics.


## B - Analysis

[01 - Coupling](#01---Coupling)&ensp;|&ensp;[02 - Strongly connected components](#02---Strongly-connected-components)&ensp;|&ensp;[03 - Network vulnerability](#03---Network-vulnerability)

Simply saying that one package depends transitively on another does not tell the whole story. A defect in one package can affect another in different ways, depending on the structure of the network.

### 01 - Coupling

In [ ]:
from olivia.model import OliviaNetwork

pypi = OliviaNetwork(r'data/pypi-2020-01-12.olv')

Let's choose a simple case study.

*spaCy* (https://spacy.io/) is an open-source software library for advanced natural language processing.

In [ ]:
pypi['spacy'].direct_dependencies()

*spaCy* employs 25 direct dependencies. These are the ones that the developers have explicitly imported, for example *requests*, a useful HTTP library. It should be clear to them that they take a risk by using *requests*. If it disappear, or include a bug or a vulnerability that their developers don't fix, the *spaCy* developers will have a problem. They would have to look for alternatives or perhaps directly implement the *requests* functionality.

However, *spaCy* indirectly depends on more packages:

In [ ]:
len(pypi['spacy'].transitive_dependencies())

So there are not 25 but 73 packages that can cause trouble.  This is a fact that many developers may be aware of, although it is maybe not generally considered seriously enough.

There is however another more elusive phenomenon. This is the fact that a single failure in a transitive dependency can affect several direct dependencies, if the latter depend in turn on the first. So *spaCy* does not depend equally on all its transitive dependencies.

**Transitive coupling** is the number of direct dependencies that would be affected by the failure of one specific transitive dependency. Observe for example the following figure, where a defect in *u* will affect 3 of the 4 direct dependencies of *v*:

<br>

![Transitive coupling](docs/img/trancoup.png "Transitive coupling")  

<br>

In [ ]:
from olivia.coupling import *

transitive_coupling(pypi,'six','spacy')

There it is. The failure of *six* does not only affect one direct dependency, but four. Which ones? We call this set the **coupling interface** of *six* over *spaCy*.

In [ ]:
coupling_interface(pypi,'six','spacy')

So let's consider that *six* changes its MIT license for a more restrictive one. Its transitional dependents in turn are forced to upgrade and ultimately *ftfy*, *html5lib*, *thinc* also change their software licenses.  If *spaCY* cannot or do not want to comply with this new model, it has to look for alternatives for *six* and also *ftfy*, *html5lib* and *thinc*. 

This change of license in *six* could also have been a shift in functionality, the removal of certain certification, a bug, a security problem... Transitive coupling is a measure of the potential effort needed to fix *spaCY* if *six* fails. Note that the sum of transitive couplings of a package with all its transitive dependants equals the IMPACT metric.

Finally, OLIVIA provides a method for computing all the transitive interfaces of transitive dependencies of a package over it. This is an indexed family of sets that we call **coupling profile**:


In [ ]:
coupling_profile(pypi,'spacy')

The frequency histogram of transitive coupling values for a package is an interesting view of the structure of dependencies:

In [ ]:
# Requires Matplotlib installed
import matplotlib.pyplot as plt
import numpy as np

cp = coupling_profile(pypi,'spacy')
ls = [len(cp[l]) for l in cp]
bins = np.arange(0, max(ls) + 1.5) - 0.5
plt.hist(ls,bins,color='black',histtype='stepfilled')
plt.xlabel('Transitive coupling')
plt.ylabel('Number of transitive dependencies')
plt.title('package: spacy')

You may also access coupling methods via *PackageInfoView*:

In [ ]:
pypi['spacy'].coupling_interface_from('six')

In [ ]:
pypi['six'].coupling_interface_to('spacy')

In [ ]:
pypi['spacy'].coupling_profile()

### 02 - Strongly connected components

An strongly connected component (SCC) is a set of packages where each one depends transitively on every one else. So a defect in any package or in any dependency of any package in the SCC could affect the whole SCC.

A simple cycle of dependencies creates an SCC, but there can be much more complicated ones.

This is an example of a simple SCC:

<br>

![SCC](docs/img/scc.png "Strongly connected component")  

<br>

As OLIVIA internally uses the precomputed structure of SCCs of the dependency network, you can iterate immediatly over SCCs: 

In [ ]:
scc_iterator = pypi.sccs()
print(next(scc_iterator))
print(next(scc_iterator))
print(next(scc_iterator))
print("...")

Each package forms its own SCC. PyPI has some small SCCs of size greater than one. These are the most interesting so OLIVIA provides a way of getting a list of SCCs in size order:

In [ ]:
pypi.sorted_clusters()[:10]

It is also possible to query which SCC a package belongs to, via PackageInfoView:

In [ ]:
pypi['indb'].scc()

It turns out that some networks have much larger SCCs

In [ ]:
maven = OliviaNetwork(r'data/maven-2020-01-12.olv')

Maven Central, the package repository for Java (https://mvnrepository.com/) has a SCC of almost a thousand packages:

In [ ]:
len(maven.sorted_clusters()[0])

### 03 - Network vulnerability

The vulnerability of the dependency network of software packages measures how sensitive the repository is to the random introduction of defects.

It is the average of some cost function over all the packages in the network, so it equals the expected cost of a random failure o defect, assuming a uniform failure probability.

We may use REACH as one of such cost functions. In this case, the vulnerability value would represent the expected number of packages potentially affected by a random failure.

The following figure shows several examples of simple networks with node values for REACH (R) and IMPACT (I) and below the corresponding REACH and IMPACT-vulnerability (preceded by the letter $\varphi$)

<br>

![Transitive coupling](docs/img/vulnerabilidad.png "Transitive coupling")  

<br>

In [ ]:
from olivia.networkmetrics import *

failure_vulnerability(pypi)

*failure_vulnerability* assumes REACH as cost function. As this model does not have a REACH cached, it is computed on the fly.  The calculation uses exact cost function values for each package in the network.  usually these functions have extremely skewed distributions, so average value is hard to sample.

PyPI has a REACH-vulnerability of about 16. This means that a random bug in a package could propagate in average to other 15 packages.

Maven is much more vulnerable to random failures, due to the presence of a significant size SCC.

In [ ]:
failure_vulnerability(maven)

In [ ]:
len(maven)

Even considering the size difference, this is much worse that PyPI. At least other network, npm (https://www.npmjs.com/), the repository of Node.js, has even bigger relative values.

Other cost functions provide different perspectives on network vulnerability:

In [ ]:
from olivia.packagemetrics import Impact

failure_vulnerability(pypi, metric=Impact)

A single random failure on PyPI causes a expected potential value of 23 individual dependencies to fix. As REACH-vulnerability could reflect the propagation of defects, IMPACT-vulnerability gives an idea of the average effort needed to recover the network.


## C - Immunization
[01 - Immunization Delta](#01---Immunization-Delta)&ensp;|&ensp;[02 - Subcritical networks](#02---Selecting-Immunization-targets---Subcritical-networks)&ensp;|&ensp;[03 - Supercritical networks](#03---Selecting-Immunization-targets---Supercritical-networks)&ensp;|&ensp;[03 - Advanced](#03---Selecting-Immunization-targets---Advanced)

The vulnerability of package repositories can be lowered if we protect (immunize) certain packages against failure and propagation of failure. For example, we could subject a package or set of packages to very strict quality controls, security audits or even contractually shield certain non-functional requirements, such as the type of license. All this comes at a cost and it would be good to know which package sets are most cost-effective for reducing the vulnerability.

### 01 - Immunization Delta


Immunization Delta is the decrease in vulnerability of a network after immunizing a given set of packages. Take for example the following figure, which shows three simple network models and possible immunization sets (black nodes). Under each option the REACH and IMPACT-vulnerability and the corresponding immunization delta are shown.

<br>

![Inmunization](docs/img/inmunizacion.png "Simple inmunization examples")  

<br>

In [ ]:
from olivia.model import OliviaNetwork
from olivia.immunization import *

pypi = OliviaNetwork(r'data/pypi-2020-01-12.olv')

In [ ]:
failure_vulnerability(pypi)

In [ ]:
immunization_delta(pypi,{'numpy','pandas','matplotlib'})

By immunizing this set of three well known Python packages, we reduce the REACH-vulnerability by 0.20, i.e we reduce  by 0.20 the expected amount of potentially affected packages by a random failure (from 15.73 to 15.53). It is not a drastic reduction, but it is not bad either, for working only on three packages of about 50000.

The default algorithm of *immunizacion_delta(...)* is called *'network'*. It computes the cost function network-wide, immunizes the target set of packages provided, rebuilds the model, computes the cost function again and returns the diference in vulnerability. In the previous code example, you may notice that the first calculation was retrieved from the model's cache, as we had computed REACH just before by calling *failure_vulnerability(pypi)*.

This does not seem to be a particularly efficient method and indeed it is not. However, it is not clear that a better technique exists for calculating the immunization delta of arbitrary sets and cost functions.


For the particular case of the REACH cost function, OLIVIA provides another algorithm for the immunization delta computation, *'analytic'*. In this case, the entire network is not processed. A mathematical formulation that only considers the packages transitively related to the target set is used to analytically calculate the vulnerability reduction associated with its immunization. This can be way faster for certain type of immunization sets, but in large complex networks it is often not the case, and the *'network'* method is time-bound and better.

The two methods are exact but there may be small differences due to rounding.

In [ ]:
target = {'networkx','spacy'}

In [ ]:
%time immunization_delta(pypi, target)

In [ ]:
%time immunization_delta(pypi, target, algorithm='analytic')

Of course you may be interested on computing immunizacion delta according to another cost metric:

In [ ]:
immunization_delta(pypi,{'numpy','pandas','matplotlib'}, cost_metric = Impact)

Notice how we needed to compute IMPACT twice. However the values for the original network are cached and will be reused in subsequent queries:

In [ ]:
immunization_delta(pypi,{'six'}, cost_metric = Impact)

### 02 - Selecting Immunization targets - Subcritical networks

OLIVIA includes tools to help locate good immunization sets in package dependency networks. 

Subcritical networks do not have many dependency cycles, and the largest strongly connected component (SCC) is small, logarithmic in relation to the size of the repository. PyPI is subcritical.

From here on we will deal with REACH-vulnerability unless we note otherwise.

¿How effective is to immunize random packages?

In [ ]:
failure_vulnerability(pypi)

In [ ]:
immunization_delta(pypi, iset_random(pypi, 20), algorithm='analytic')

*iset_...* methods from *olivia.immunization* produce sets of packages intended as immunization targets. *iset_random(...)* is really more like a baseline tool, selecting uniformly arbitrary ones. The immunization of 20 random packages does not seem to do much for decreasing the vulnerability (-0.0066 out of 15.73 in this case)

In [ ]:
immunization_delta(pypi, iset_random(pypi, 1000), algorithm='analytic')

Although appreciable, the immunization of 1000 random packages does not look spectacular either. *iset_random(...)* also provides an option to make an indirect selection, that is select randomly chosen dependencies of randomly chosen packages. This is a well known immunization tactic from Network Science, often used to exemplify advanced vaccination strategies. In this case it is also much better that pure random selection.

In [ ]:
immunization_delta(pypi, iset_random(pypi, 20, indirect=True), algorithm='analytic')

If we look at it, this indirect strategy is in fact a probabilistic approach to selection based on the number of dependents. Here we can just try the exact approach:

In [ ]:
target = pypi.get_metric(DependentsCount).top(20)
target

In [ ]:
immunization_delta(pypi, {l[0] for l in target}, algorithm='analytic')

*iset_naive_ranking(...)* is a shortcut to select a set of nodes based simply on a ranking over a metricStats object:

In [ ]:
immunization_delta(pypi, 
                   iset_naive_ranking(20, pypi.get_metric(DependentsCount)),
                   algorithm='analytic')

Choosing the 20 packages with higher count of direct dependants is even better that indirect selection. Now we achieve a vulnerability reduction of near a third, acting only on 20 out of 50.000 packages.

If we want to reduce REACH-vulnerability, a simple idea to consider could be to immunize the nodes with the greatest REACH value. It is somewhat surprising that this strategy is worse than the dependants count based one:

In [ ]:
immunization_delta(pypi, 
                   iset_naive_ranking(20, pypi.get_metric(Reach)),
                   algorithm='analytic')

In addition to ranking methods, OLIVIA provides some specific techniques to find good targets. *iset_delta_set_reach(...)* computes a set of nodes that meets certain theoretical constraints on REACH and SURFACE values to ensure that it contains the best single node to immunize from the network. It is possible that the set contains other good targets.

The size of the delta set is given by the algorithm. For PyPI it is really small, of only 18 packages.

In [ ]:
delta_set = iset_delta_set_reach(pypi)
print(delta_set)

In [ ]:
immunization_delta(pypi, delta_set, algorithm='analytic')

This is our best result so far, a 6.40 out of 15.73 (41%) decrease in vulnerability by immunizing 18 packages.

Of course we can also rank the results within the delta set for finding smaller sets:

In [ ]:
smaller = iset_naive_ranking(4, pypi.get_metric(DependentsCount), subset = delta_set)
print(smaller)
immunization_delta(pypi, smaller, algorithm='analytic')

### 03 - Selecting Immunization targets - Supercritical networks

Supercritical packet dependency networks are those that contain a strongly connected (SCC) component of significant size. Maven is one of them.

In [ ]:
maven = OliviaNetwork(r'data/maven-2020-01-12.olv')

Supercritical networks are much more vulnerable, because the big SCC contributes greatly to the propagation of defects.

In [ ]:
failure_vulnerability(maven)

Lets see what happens if we immunize the delta set for Maven:

In [ ]:
maven_delta_set = iset_delta_set_reach (maven)
immunization_delta(maven, maven_delta_set)

We reduced the vulnerability of Maven by 1800!. This means that before immunization, the failure of a single package could affect an average of 1805 other packages. After immunization, this number is 5 (five packages).

However, there is a catch.

In [ ]:
len(maven_delta_set)

In supercrítical networks, the delta set is larger. Here it represent aproximately 5% of the network (was 0.04% in PyPI). In fact, the delta set is not very useful in supercritical networks, as it usually contains the largest SCC, which is the real cause of the high vulnerability.

*iset_sap(...)* computes an immunization set by detecting the strong articulation points (SAP) of the biggest SCC in the network. SAPs are packages whose immunization is likely to contribute to the break-up of the SCC, reducing its ability to spread defects.

In [ ]:
sap = iset_sap(maven)

immunization_delta(maven, sap)

In [ ]:
len(sap)

SAP is much smaller (0.28%), and still achieves a reduction in vulnerability of near 94%

Note that here we have not used the 'analityc' method for the calculation of the immunization delta, since sets are larger and it could be really slow.

Ranking on the SAP set gives us a tool for finding smaller sets:

In [ ]:
immunization_delta(maven, iset_naive_ranking(100, maven.get_metric(DependentsCount), subset=sap))

Notice that the SAP approach is completely innefective in subcritical networks

In [ ]:
immunization_delta(pypi, iset_sap(pypi), algorithm='analytic')

Even the immunization of the largest SCC in full has a negligible effect on the network's vulnerability

In [ ]:
immunization_delta(pypi, pypi.sorted_clusters()[0], algorithm='analytic')

### 04 - Selecting Immunization targets - Advanced

Calculating the immunization delta is computationally expensive and the problem of selecting an optimal immunization set of a given size is mathematically intractable for large repositories.

In this guide we have used PyPI and Maven, but there are much larger networks, such as npm with over a million packages.

We will now give some ideas of additional techniques that can be used for the heuristic exploration of the problem.

#### Manipulating immunization sets

Set operations may be interesting to facilitate the experimental search for good immunization sets.

Packages both in PyPI delta set and in top 100 by dependentsCount:

In [ ]:
delta_set & iset_naive_ranking(100, pypi.get_metric(DependentsCount))

In [ ]:
immunization_delta(pypi, {'boto3',
                         'configparser',
                         'cryptography',
                         'jsonschema',
                         'packaging',
                         'pbr',
                         'pytest',
                         'requests',
                         'setuptools',
                         'wheel'}, algorithm='analytic')

That approach is much better than simply selecting the top 10 packages by DependentsCount:

In [ ]:
immunization_delta(pypi,
                   iset_naive_ranking(10, pypi.get_metric(DependentsCount)), 
                   algorithm='analytic')

#### Ranking by compound metrics

Since arithmetic operations can be performed with MetricStats objects, it is easy to employ ranking functions with compound metrics. For example, REACH*SURFACE is a theoretical upper bound of immunization delta.

In [ ]:
upper = iset_naive_ranking(50, pypi.get_metric(Reach)*pypi.get_metric(Surface))
immunization_delta(pypi, upper, algorithm='analytic')

In fact, in this case we can get exactly the PyPI delta set, using this technique with the added benefit of being able to regulate the number of elements in the set.

In [ ]:
len(delta_set)

In [ ]:
upper = iset_naive_ranking(18, pypi.get_metric(Reach)*pypi.get_metric(Surface))

In [ ]:
# void set
delta_set-upper

#### Brute force and Greedy selection

Once we get small enough candidate sets, we could try brute-force and greedy approaches for finding smaller sets. 

Considering the PyPI delta set, let's see which package individually contributes most to vulnerability reduction:

In [ ]:
[(i, immunization_delta(pypi, {i}, algorithm='analytic')) for i in delta_set]

It should be noted that, as the sets of dependants and transitional dependencies are interrelated, the immunization delta is not additive. For example, if you take out *'importlib-metadata'* from the delta set, immunization delta decreases only by a small amount:

In [ ]:
immunization_delta(pypi, delta_set-{'importlib-metadata'}, algorithm='analytic')

#### Using other centrality measures

Common measures of centrality in the field of network science, or more custom or specific ones, can be used to select immunization sets. You may need to narrow down the set of candidates to  make the computation feasible.

For example, using betweenness centrality on the SAP set gives the best results we have found for small immunization sets in Maven:

In [ ]:
import networkx as nx

# Using the NetworkX implementation of betweenness centrality
# You can access the full NetworkX network underlying the repository using OliviaModel.network.
# Here we restrict the centrality computation to the subgraph induced by the SAP set:
sap_betweenness = nx.betweenness_centrality(maven.network.subgraph(sap))

In [ ]:
from olivia.packagemetrics import MetricStats

# Build a MetricStats object to use in iset_naive_ranking
# MetricStats constructor admits a dictionary of values such as those produced by NetworkX
sap_betweenness = MetricStats(sap_betweenness)

In [ ]:
immunization_delta(maven, iset_naive_ranking(10, sap_betweenness))

Follows a visual representation of this technique. The drawn portion (981 nodes) is the biggest SCC of Maven , representing about 0.8% of the network. 

The highlighted white nodes correspond to the SAP set (351 nodes), whose immunization reduces by 93.9% the REACH-vulnerability of the network as a whole. The blue highlights are the 10 SAP with the highest out-degree centrality (number of dependants) and the red ones the 10 with the highest betweenness. By immunizing these sets, a reduction in network vulnerability of 13% and 26% respectively is achieved.
The representation omits the direction of the arcs and uses the Kamada-kawai algorithm.

<br>

![Maven SCC](docs/img/maven.png "SAP immunization of Maven with betweenness ranking")  

<br>